### Generate image embeddings

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
file_path = '/kaggle/input/netflix-rotten-tomatoes-metacritic-imdb/netflix-rotten-tomatoes-metacritic-imdb.csv'
data = pd.read_csv(file_path)
data.head(3)

In [ ]:
print(data.shape)
print(data.isnull().sum())

In [ ]:
from PIL import Image
import requests
#import cv2
import matplotlib.pyplot as plt
#from keras.utils import img_to_array

def url_to_array(poster_links): 
    posters = []
    i = 0
    idx = []
    for url in poster_links:
        if i%100 == 0 :
            print(i)
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            img = Image.open(response.raw)
            new_size = (160,240)
            img = img.resize(new_size)
            #img_arr = img_to_array(img)
            #img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            img_arr = np.asarray(img,dtype="uint8")
            posters.append(img_arr)
            idx.append(i)
        i = i + 1
    return idx,posters

In [ ]:
# Retrieve images from ULRs
# Convert from image format to array representation: (height, width, channel)
idx_ls, poster_ls = url_to_array(data.Image)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400


In [ ]:
poster = pd.DataFrame({'index':idx_ls,'poster': poster_ls})
print(poster.shape)
poster.head()

(15453, 2)


,index,poster
0,0,"[[[36, 19, 9], [36, 18, 8], [35, 16, 9], [36, ..."
1,1,"[[[101, 127, 150], [102, 128, 151], [103, 129,..."
2,2,"[[[49, 77, 101], [52, 80, 104], [53, 81, 105],..."
3,3,"[[[228, 229, 234], [228, 229, 234], [228, 229,..."
4,4,"[[[213, 213, 213], [213, 213, 213], [213, 213,..."


In [ ]:
poster.iloc[0,1].shape

(240, 160, 3)

In [ ]:
poster.to_csv('poster_array.csv',index=False)

In [ ]:
import torch
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.models import Model,Sequential
from keras.layers import Flatten, Dropout, MaxPooling2D,GlobalAveragePooling2D, Concatenate

base_model = InceptionV3(input_shape = (240, 160, 3), include_top = False, weights = 'imagenet')
base_model.trainable = False
x = base_model.layers[-1].output
x = GlobalAveragePooling2D()(x)
output = Flatten()(x)

# define new model
model = Model(inputs=base_model.inputs, outputs=output)
# summarize
#print(model.summary())

87910968/87910968 [==============================] - 4s 0us/step


In [ ]:
def img_to_embedding(df_posters):
    embedding_ls = []
    idx = []
    
    for i in range(0,df_posters.shape[0]):
        # reshape data
        image = df_posters.iloc[i,1]
        try:
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            img_preprocessed = preprocess_input(image)
            features = model.predict(img_preprocessed)
            embedding_ls.append(features)
            idx.append(df_posters.iloc[i,0])
        except Exception as error: 
            print("An exception occurred on line",i)
            print(error)

        if i%100 == 0:
            print(i)
            
    return idx, embedding_ls

In [ ]:
# Generate image embeddings using the pre-trained InceptionV3 model
index,features = img_to_embedding(poster)

In [ ]:
# Fail to compute features for black & white images as they only have 2 channels 
# print(data.Image[418]) 
# poster.iloc[418,:]

In [ ]:
poster_features = np.vstack(features)
poster_features.shape

(15435, 2048)

In [ ]:
# Save the results
np.savetxt("poster_index.csv", index, delimiter=",") #'index' refers the corresponding index of a poster in the original dataset, which can be used to filter rows with valid url links.
np.savetxt("poster_features.csv", poster_features, delimiter=",") # 2048 dimensional image embeddings 

### To use image features directly, load the following files:

In [ ]:
poster_index = np.loadtxt('poster_index.csv',delimiter=',')
print(poster_index.shape)

In [ ]:
# May take several mins to load
poster_features = np.loadtxt('poster_features.csv',delimiter=',')
poster_features.shape

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
poster_index = np.loadtxt('/content/drive/My Drive/Colab Notebooks/poster_index.csv',delimiter=',')
print(poster_index.shape)

In [ ]:
# May take several mins to load
poster_features = np.loadtxt('/content/drive/My Drive/Colab Notebooks/poster_features.csv',delimiter=',')
poster_features.shape

In [ ]:
index_to_keep = [int(i) for i in poster_index]

In [ ]:
filtered_data = data.drop(index=data.index.difference(index_to_keep))
filtered_data.shape

In [ ]:
poster_features = pd.DataFrame(poster_features)
poster_features.shape

In [ ]:
filtered_data.reset_index(drop=True, inplace=True)
poster_features.reset_index(drop=True, inplace=True)

In [ ]:
merged_data = pd.concat([filtered_data,poster_features],axis=1)

### Packaged image processing for pipeline

In [12]:
import numpy as np 
import pandas as pd 

In [7]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/') 

Mounted at /content/drive


In [8]:
from PIL import Image

def url_to_array(file_path): 
  img = Image.open(file_path)
  new_size = (160,240)
  img = img.resize(new_size)
  img_arr = np.asarray(img,dtype="uint8")

  return img_arr

In [9]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

def img_to_embedding(image,model):
  try:
      image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
      img_preprocessed = preprocess_input(image)
      features = model.predict(img_preprocessed)
  except Exception as error: 
      print("An exception occurred")
      print(error)
  
  return features

In [10]:
def getImgFeatures(file_path,model):
  img_arr = url_to_array(file_path)
  features = img_to_embedding(img_arr,model)
  # Convert array to dataframe and assign column names
  col_name = ['poster_'+str(i+1) for i in range(features.shape[1])]
  features_df = pd.DataFrame(features,columns=col_name)
  # Trim features for popularity task 
  selected_cols_popularity = ['poster_1179','poster_1057','poster_278','poster_1911','poster_711','poster_1074','poster_197','poster_856','poster_1876','poster_236']
  df_popularity = features_df[selected_cols_popularity]
  # Trim features for quality task
  selected_cols_quality = ['poster_1653','poster_143','poster_1594','poster_1592','poster_1070','poster_1623','poster_1011','poster_23','poster_1587','poster_451','poster_1586']
  df_quality = features_df[selected_cols_quality]
  
  return df_popularity,df_quality

In [13]:
import pickle
# Load the saved model upon loading the webpage
model_path = '/content/drive/My Drive/Colab Notebooks/image_model.pkl'
with open(model_path, "rb") as f:
  loaded_model = pickle.load(f)

In [ ]:
# Call the following per user request:
file_path = "/content/drive/My Drive/Colab Notebooks/Pika7U.jpg"
features_popularity, features_quality = getImgFeatures(file_path,loaded_model)
print(features_popularity.shape)
print(features_quality.shape)

In [ ]:
features_popularity

In [ ]:
features_quality